# This notebook is for running the code to classify AuxTel mount errors.
The code works by fitting a set of spline knots to the FFT of the mount errors, then using a neural netwrok based classifier to classify the different types of mount errors.

Craig Lage - 02Jan24

In [ ]:
# The list below is the set of dayObs used to build a list of errors to be manually classified
dayObs_list = [20221110, 20221212, 20230118, 20230216, \
               20220315, 20230511, 20230817, 20231107, 20231113, 20231121, 20231128, 20231129, 20231130]

limit = 0.25 # This is the limit of image_RMS above which it is considered a mount error.


In [ ]:
import nest_asyncio
nest_asyncio.apply()
import sys, time, os, asyncio
from datetime import datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler

import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

script_path = "/home/c/cslage/u/AuxTel/scripts"
sys.path.append(script_path)
from Classifying import calculateFFTKnots

# The first step is to build a list of mount failures to be manually classified
## These will be used to train the neural network

In [ ]:
client = EfdClient('idf_efd')
butler = Butler('/repo/embargo', collections="LATISS/raw/all")

Mount_FFT_Dict = {}
for dayObs in dayObs_list[3:4]:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        if record.observation_type not in ['bias', 'flat', 'dark']:
            exposureList.append(record.id)
    exposureList = sorted(exposureList)

    for expId in exposureList:
        try:
            dataId = {'detector':0, 'exposure':expId}
            result = calculateFFTKnots(dataId, butler, client, limit=limit)
            if result:
                [tot_rms, fftKnots, timebaseFlag] = result
                resDict = {}
                resDict['Cause'] = None
                resDict['RMS'] = tot_rms
                resDict['FFT_knots'] = fftKnots
                Mount_FFT_Dict[expId] = resDict
                print(f"Finished {expId}")
        except:
            continue
    outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Knots_02Jan24.pkl', 'wb')

    pkl.dump(Mount_FFT_Dict,outfile)
    outfile.close()
    print(f"Finished {dayObs}")


# The next step is to manually classify the fails.

This code will bring up the RubinTV mount plot one-by-one and you can enter the classification.  If you enter STOP, it will save what you have done so you can take a break.
# Note:
I ran this on the jupyter notebook on my laptop (not at the RSP), so it has control of the webbrowser.

In [ ]:
import webbrowser
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Knots_02Jan24.pkl', 'rb')
Mount_FFT_Dict = pkl.load(infile)
Mount_FFT_Dict_Classified = Mount_FFT_Dict.copy()
infile.close()
outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Knots_02Jan24_Classified.pkl', 'wb')
pkl.dump(Mount_FFT_Dict_Classified,outfile)
outfile.close()

causes = ['OSC', 'WIN', 'CRA', 'SHU']

print(causes)
for key in Mount_FFT_Dict_Classified.keys():
    expId = int(key)
    dayObs = int(expId / 100000)
    dateTime = datetime.strptime(str(dayObs), "%Y%m%d")
    year = dateTime.year
    month = dateTime.month
    day = dateTime.day
    seqNum = int((expId - 1000000000 * year - 10000000 * month - 100000 * day))

    if Mount_FFT_Dict_Classified[key]['Cause'] is None:
        webbrowser.open_new(f'https://roundtable.lsst.codes/rubintv/summit/auxtel/mount/event/{year}-{month:02}-{day:02}/{seqNum}')
        cause = input(f"Classification of {key}")
        if cause == 'STOP':
            break
        else:
            Mount_FFT_Dict_Classified[key]['Cause'] = cause

outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Knots_02Jan24_Classified.pkl', 'wb')
pkl.dump(Mount_FFT_Dict_Classified,outfile)
outfile.close()

# The next step is to build the neural network based classifier

In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict_Knots_Classified.pkl', 'rb')
Mount_FFT_Dict_Knots_Classified = pkl.load(infile)
infile.close()
causes = ['OSC', 'WIN', 'CRA', 'SHU']

data = []
labels = []
for key in Mount_FFT_Dict_Knots_Classified.keys():
    if Mount_FFT_Dict_Knots_Classified[key]['Cause'] in ['OTH', 'TIM']:
        continue
    data.append(np.array(Mount_FFT_Dict_Knots_Classified[key]['FFT_knots']))
    for i, cause in enumerate(causes):
        if Mount_FFT_Dict_Knots_Classified[key]['Cause'] == cause:
            labels.append(i)
print(len(labels))

In [ ]:
datasets = train_test_split(data, 
                            labels,
                            test_size=0.2)

train_data, test_data, train_labels, test_labels = datasets

### This gave a 96% success rate on the test set

In [ ]:
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(80,80,40,20), \
                    random_state=1, max_iter=1000000, learning_rate='invscaling', \
                     n_iter_no_change=10000)
verbose=10,
clf.fit(train_data, train_labels) 
predictions_train = clf.predict(train_data)
predictions_test = clf.predict(test_data)
train_score = accuracy_score(predictions_train, train_labels)
print("Score on train data: ", train_score)
test_score = accuracy_score(predictions_test, test_labels)
print("Score on test data: ", test_score)

# Now re-build it with all of the data.

In [ ]:
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(80,80,40,20), \
                    random_state=1, max_iter=1000000, learning_rate='invscaling', \
                     n_iter_no_change=10000)
verbose=10,
clf.fit(data, labels) 
predictions_train = clf.predict(data)
train_score = accuracy_score(predictions_train, labels)
print("Score on train data: ", train_score)
outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Knots_Based_Classifier_No_TIM_28Dec23.pkl', 'wb')
pkl.dump(clf,outfile)
outfile.close()

# Now we run it on all of 2023

In [ ]:
# This allows a restart if something crashes
restart = False
from lsst_efd_client import EfdClient
#client = EfdClient("usdf_efd", db_name="efd-s2022-08-10") # Before 2022-08-10
#client = EfdClient('idf_efd') # Before 20231211                                                                                                   
client = EfdClient('usdf_efd') # 20231211 and after                                                                                                
#butler = Butler('/repo/main', collections="LATISS/raw/all")
butler = Butler('/repo/embargo', collections="LATISS/raw/all")
# Get the classifier                                                                                                                               
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Knots_Based_Classifier_No_TIM_28Dec23.pkl', 'rb')
clf = pkl.load(infile)
infile.close()

if restart:
    startDay = 20220912 # Where to restart                                                                                                         
    # Read in what is already done:                                                                                                                
    infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_2022_07Jan24.pkl', 'rb')
    Mount_Errors_Classified_Dict = pkl.load(infile)
    infile.close()
else:
    startDay = 20240301
    Mount_Errors_Classified_Dict = {}
endDay = 20240303

causes = ['OSC', 'WIN', 'CRA', 'SHU']
fullCauses = ['UNSURE', 'OSC', 'WIN', 'CRA', 'TIM', 'SHU', 'GOOD']

dayObs = startDay
while dayObs <= endDay:
    records = butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs)
    print(dayObs, records.count())
    if records.count() < 200:
        dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                       
        continue
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        if record.observation_type not in ['bias', 'flat', 'dark'] and record.exposure_time > 2.0:
            exposureList.append(record.id)
    exposureList = sorted(exposureList)
    dayObsDict = dict((cause,[]) for cause in fullCauses)
    print(dayObs, len(exposureList))
    for expId in exposureList:
        try:
            seqNum = int(expId - dayObs * 1E5)
            dataId = {'detector':0, 'exposure':expId}
            result = calculateFFTKnots(dataId, butler, client, limit=limit)
            if result:
                [tot_rms, knotList, timebaseFlag] = result
                if knotList:
                    # This means the tot_rms was > limit and a knotList was created                                                                
                    if timebaseFlag:
                        dayObsDict['TIM'].append(seqNum)
                    else:
                        prob = np.max(clf.predict_proba([knotList])[0])
                        if prob < 0.75:
                            dayObsDict['UNSURE'].append(seqNum)
                        else:
                            cause = causes[clf.predict([knotList])[0]]
                            dayObsDict[cause].append(seqNum)
                else:
                    # This means the tot_rms was less than the limit                                                                               
                    dayObsDict['GOOD'].append(seqNum)
            print(f"{expId} finished successfully")
            sys.stdout.flush()
        except:
            print(f"{expId} failed")
            sys.stdout.flush()
            continue
    Mount_Errors_Classified_Dict[dayObs] = dayObsDict
    print(f"Finished {dayObs}")
    sys.stdout.flush()
    dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                           
    outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_2024_03Mar24.pkl', 'wb')
    pkl.dump(Mount_Errors_Classified_Dict,outfile)
    outfile.close()
                            

# Now we run it on all of 2024

In [ ]:
# This allows a restart if something crashes
restart = False
from lsst_efd_client import EfdClient
#client = EfdClient("usdf_efd", db_name="efd-s2022-08-10") # Before 2022-08-10
#client = EfdClient('idf_efd') # Before 20231211                                                                                                   
client = EfdClient('usdf_efd') # 20231211 and after                                                                                                
#butler = Butler('/repo/main', collections="LATISS/raw/all")
butler = Butler('/repo/embargo_old', collections="LATISS/raw/all")
# Get the classifier                                                                                                                               
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Knots_Based_Classifier_No_TIM_28Dec23.pkl', 'rb')
clf = pkl.load(infile)
infile.close()

if restart:
    startDay = 20220912 # Where to restart                                                                                                         
    # Read in what is already done:                                                                                                                
    infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_2024_21Jan25.pkl', 'rb')
    Mount_Errors_Classified_Dict = pkl.load(infile)
    infile.close()
else:
    startDay = 20240129
    Mount_Errors_Classified_Dict = {}
endDay = 20241016

causes = ['OSC', 'WIN', 'CRA', 'SHU']
fullCauses = ['UNSURE', 'OSC', 'WIN', 'CRA', 'TIM', 'SHU', 'GOOD']

dayObs = startDay
while dayObs <= endDay:
    records = butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs)
    print(dayObs, records.count())
    if records.count() < 200:
        dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                       
        continue
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        if record.observation_type not in ['bias', 'flat', 'dark'] and record.exposure_time > 2.0:
            exposureList.append(record.id)
    exposureList = sorted(exposureList)
    dayObsDict = dict((cause,[]) for cause in fullCauses)
    print(dayObs, len(exposureList))
    for expId in exposureList:
        try:
            seqNum = int(expId - dayObs * 1E5)
            dataId = {'detector':0, 'exposure':expId}
            result = calculateFFTKnots(dataId, butler, client, limit=limit)
            if result:
                [tot_rms, knotList, timebaseFlag] = result
                if knotList:
                    # This means the tot_rms was > limit and a knotList was created                                                                
                    if timebaseFlag:
                        dayObsDict['TIM'].append(seqNum)
                    else:
                        prob = np.max(clf.predict_proba([knotList])[0])
                        if prob < 0.75:
                            dayObsDict['UNSURE'].append(seqNum)
                        else:
                            cause = causes[clf.predict([knotList])[0]]
                            dayObsDict[cause].append(seqNum)
                else:
                    # This means the tot_rms was less than the limit                                                                               
                    dayObsDict['GOOD'].append(seqNum)
            print(f"{expId} finished successfully")
            sys.stdout.flush()
        except:
            print(f"{expId} failed")
            sys.stdout.flush()
            continue
    Mount_Errors_Classified_Dict[dayObs] = dayObsDict
    print(f"Finished {dayObs}")
    sys.stdout.flush()
    dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                           
    outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_2024_21Jan25.pkl', 'wb')
    pkl.dump(Mount_Errors_Classified_Dict,outfile)
    outfile.close()
                            

# Now we run it on all of 2025

In [ ]:
# This allows a restart if something crashes
restart = False
from lsst_efd_client import EfdClient
#client = EfdClient("usdf_efd", db_name="efd-s2022-08-10") # Before 2022-08-10
#client = EfdClient('idf_efd') # Before 20231211                                                                                                   
client = EfdClient('usdf_efd') # 20231211 and after                                                                                                
butler = butlerUtils.makeDefaultButler("LATISS")
# Get the classifier                                                                                                                               
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Knots_Based_Classifier_No_TIM_28Dec23.pkl', 'rb')
clf = pkl.load(infile)
infile.close()

if restart:
    startDay = 20220912 # Where to restart                                                                                                         
    # Read in what is already done:                                                                                                                
    infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_2025_21Jan25.pkl', 'rb')
    Mount_Errors_Classified_Dict = pkl.load(infile)
    infile.close()
else:
    startDay = 20250107
    Mount_Errors_Classified_Dict = {}
endDay = 20250121

causes = ['OSC', 'WIN', 'CRA', 'SHU']
fullCauses = ['UNSURE', 'OSC', 'WIN', 'CRA', 'TIM', 'SHU', 'GOOD']

dayObs = startDay
while dayObs <= endDay:
    records = butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs)
    print(dayObs, records.count())
    if records.count() < 200:
        dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                       
        continue
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        if record.observation_type not in ['bias', 'flat', 'dark'] and record.exposure_time > 2.0:
            exposureList.append(record.id)
    exposureList = sorted(exposureList)
    dayObsDict = dict((cause,[]) for cause in fullCauses)
    print(dayObs, len(exposureList))
    for expId in exposureList:
        try:
            seqNum = int(expId - dayObs * 1E5)
            dataId = {'detector':0, 'exposure':expId}
            result = calculateFFTKnots(dataId, butler, client, limit=limit)
            if result:
                [tot_rms, knotList, timebaseFlag] = result
                if knotList:
                    # This means the tot_rms was > limit and a knotList was created                                                                
                    if timebaseFlag:
                        dayObsDict['TIM'].append(seqNum)
                    else:
                        prob = np.max(clf.predict_proba([knotList])[0])
                        if prob < 0.75:
                            dayObsDict['UNSURE'].append(seqNum)
                        else:
                            cause = causes[clf.predict([knotList])[0]]
                            dayObsDict[cause].append(seqNum)
                else:
                    # This means the tot_rms was less than the limit                                                                               
                    dayObsDict['GOOD'].append(seqNum)
            print(f"{expId} finished successfully")
            sys.stdout.flush()
        except:
            print(f"{expId} failed")
            sys.stdout.flush()
            continue
    Mount_Errors_Classified_Dict[dayObs] = dayObsDict
    print(f"Finished {dayObs}")
    sys.stdout.flush()
    dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                           
    outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_2025_21Jan25.pkl', 'wb')
    pkl.dump(Mount_Errors_Classified_Dict,outfile)
    outfile.close()
                            

In [ ]:
t_start = Time('2024-03-04T12:00:00', scale='utc')
t_end = Time('2024-03-04T23:59:00', scale='utc')

mount_position = await client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders",['*'],t_start, t_end)
len(mount_position)

# The following cells are for analyzing the classified data

## This cell will print the list of seqNums by cause

In [ ]:
dayObs = 20231128
fullCauses = ['UNSURE', 'OSC', 'WIN', 'CRA', 'TIM', 'SHU', 'GOOD'] 
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_29Dec23.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()
for cause in fullCauses:
    print(cause, Mount_Errors_Classified_Dict[dayObs][cause])

## This plots the summary for the year

In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_2025_21Jan25.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()
dates = Mount_Errors_Classified_Dict.keys()
print(dates)



In [ ]:
fullCauses = ['UNSURE', 'OSC', 'WIN', 'CRA', 'TIM', 'SHU', 'GOOD'] 

infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_2025_21Jan25.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()

plotColors = ['gray', 'red', 'blue', 'violet', 'yellow', 'cyan', 'green']
dates = Mount_Errors_Classified_Dict.keys()
plotDates = []
plotList = []
xticks = []

skipDates = 1
# Somehow some empty lists got in
# This cleans them out
badDates = []
for date in dates:
    if len(Mount_Errors_Classified_Dict[date]['GOOD']) < 5:
        badDates.append(date)
for date in badDates:
    del Mount_Errors_Classified_Dict[date]
        
dates = Mount_Errors_Classified_Dict.keys()
data = np.zeros([len(dates), len(fullCauses)])
for i, date in enumerate(dates):
    for j, cause in enumerate(fullCauses):
        data[i,j] = len(Mount_Errors_Classified_Dict[date][cause])
for i, date in enumerate(dates):
    sum = data[i,:].sum()
    if i % skipDates == 0:
        xticks.append(str(date))
    else:
        xticks.append("")
    plotDates.append(str(date))
    for j, cause in enumerate(fullCauses):
        data[i,j] /= sum
        data[i,j] *= 100.0

for j, cause in enumerate(fullCauses):
    plotList.append(data[:,j])


fig, ax = plt.subplots(figsize=(10,8))
ax.set_title("AuxTel types of mount fails - 2025", fontsize=18)
ax.stackplot(plotDates, plotList, labels=fullCauses, colors=plotColors)
ax.legend(loc='upper left')
ax.set_ylabel("Percent")
ax.set_xticks(ticks=plotDates,labels=xticks, rotation=90)
plt.savefig('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Fails_Classified_2025.png')

## This plots the wind speed correlation

In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_29Dec23.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()
dates = Mount_Errors_Classified_Dict.keys()

fullCauses = ['UNSURE', 'OSC', 'WIN', 'CRA', 'TIM', 'SHU', 'GOOD'] 
windSpeeds = []
windFails = []
client = EfdClient('idf_efd') 
for date in dates:
    try:
        dayObs = int(date)
        dateTime = datetime.strptime(str(dayObs), "%Y%m%d")
        dayObsDate = f"{dateTime.year}-{dateTime.month}-{dateTime.day}"
        nextDayObs = calcNextDay(dayObs)
        dateTime = datetime.strptime(str(nextDayObs), "%Y%m%d")
        nextDayObsDate = f"{dateTime.year}-{dateTime.month}-{dateTime.day}"
        start = Time(f"{dayObsDate} 23:00:00Z", scale='utc')
        end = Time(f"{nextDayObsDate} 08:00:00Z", scale='utc')
        maxSpeed = await client.select_time_series('lsst.sal.ESS.airFlow', \
                                                ['maxSpeed'],  start, end, index=301)
        windSpeed = np.median(maxSpeed['maxSpeed'].values)
        total = 0
        for cause in fullCauses:
            total += len(Mount_Errors_Classified_Dict[date][cause])
            if cause == 'WIN':
                windFail = len(Mount_Errors_Classified_Dict[date][cause])
        windFail = windFail / total * 100.0
        windSpeeds.append(windSpeed)
        windFails.append(windFail)
    except:
        continue

print(len(windSpeeds), len(windFails))
fig = plt.figure()
plt.scatter(windSpeeds, windFails)
plt.title("Mount fails due to wind jitter vs median wind speed", fontsize=16)
plt.ylabel("Percent fails due to wind jitter")
plt.xlabel("Median wind speed (m/s)")
plt.xlim(0,15)
plt.savefig('/home/c/cslage/u/AuxTel/mount_classifier/Wind_Fails_vs_Wind_Speed_2023.png')

## This cell allows plotting the spline knots of a given expId

In [ ]:
#client = EfdClient('idf_efd')
#butler = Butler('/repo/embargo', collections="LATISS/raw/all")
expId = 2023113000629
dataId = {'detector':0, 'exposure':expId}
fig = plt.figure(figsize=(10,5))
plt.suptitle(f"FFT SPline Knots {expId}")
result = calculateFFTKnots(dataId, butler, client, limit=limit, fig=fig)
plt.savefig(f'/home/c/cslage/u/AuxTel/mount_classifier/Spline_Knots_{expId}.png')

In [ ]:
butler = Butler('/repo/embargo', collections="LATISS/raw/all")

In [ ]:
client = EfdClient("usdf_efd", db_name="efd-s2022-08-10")
butler = Butler('/repo/main', collections="LATISS/raw/all")
limit = 0.25
expId = 2021090900500
dataId = {'detector':0, 'exposure':expId}
result = calculateFFTKnots(dataId, butler, client, limit=limit)
print(result)

In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_2021_07Jan24.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()
for key in Mount_Errors_Classified_Dict.keys():
    print(key, len(Mount_Errors_Classified_Dict[key]['GOOD']))

In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_2022_07Jan24.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()
for key in Mount_Errors_Classified_Dict.keys():
    print(key, len(Mount_Errors_Classified_Dict[key]['GOOD']))

# Now combine the three years

In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_2021_07Jan24.pkl', 'rb')
Mount_Errors_Classified_Dict_2021 = pkl.load(infile)
infile.close()
print(len(Mount_Errors_Classified_Dict_2021))
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_2022_07Jan24.pkl', 'rb')
Mount_Errors_Classified_Dict_2022 = pkl.load(infile)
infile.close()
print(len(Mount_Errors_Classified_Dict_2022))
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_29Dec23.pkl', 'rb')
Mount_Errors_Classified_Dict_2023 = pkl.load(infile)
infile.close()
print(len(Mount_Errors_Classified_Dict_2023))
Mount_Errors_Classified_Dict_All = {}
for key in Mount_Errors_Classified_Dict_2021.keys():
    if len(Mount_Errors_Classified_Dict_2021[key]['GOOD']) == 0:
        continue
    else:
        Mount_Errors_Classified_Dict_All[key] = Mount_Errors_Classified_Dict_2021[key]
for key in Mount_Errors_Classified_Dict_2022.keys():
    if len(Mount_Errors_Classified_Dict_2022[key]['GOOD']) == 0:
        continue
    else:
        Mount_Errors_Classified_Dict_All[key] = Mount_Errors_Classified_Dict_2022[key]
for key in Mount_Errors_Classified_Dict_2023.keys():
    if len(Mount_Errors_Classified_Dict_2023[key]['GOOD']) == 0:
        continue
    else:
        Mount_Errors_Classified_Dict_All[key] = Mount_Errors_Classified_Dict_2023[key]
        
print(len(Mount_Errors_Classified_Dict_All))
outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_All_07Jan24.pkl', 'wb')
pkl.dump(Mount_Errors_Classified_Dict_All,outfile)
outfile.close()


In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_All_07Jan24.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()
for key in Mount_Errors_Classified_Dict.keys():
    print(key, len(Mount_Errors_Classified_Dict[key]['GOOD']))

In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_All_07Jan24.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()

obsId = 20221027
print(Mount_Errors_Classified_Dict[obsId]['GOOD'])


In [ ]:
fullCauses = ['UNSURE', 'OSC', 'WIN', 'CRA', 'TIM', 'SHU', 'GOOD']
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_All_07Jan24.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()

obsId = 20221027
for cause in fullCauses:
    print(cause, Mount_Errors_Classified_Dict[obsId][cause])
